# QSAR of Amyloid Beta (Abeta) - Computational Drug Discovery #
***
## Part 1 - Data Queries & Cleaning ##
Using the ChEMBL database via their webresourceclient API

In [26]:
! pip install Django chembl_webresource_client

  Obtaining dependency information for Django from https://files.pythonhosted.org/packages/7f/9e/fc6bab255ae10bc57fa2f65646eace3d5405fbb7f5678b90140052d1db0f/Django-4.2.4-py3-none-any.whl.metadata
  Obtaining dependency information for asgiref<4,>=3.6.0 from https://files.pythonhosted.org/packages/9b/80/b9051a4a07ad231558fcd8ffc89232711b4e618c15cb7a392a17384bbeef/asgiref-3.7.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 966.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 31.5 MB/s eta 0:00:0000:0100:01


In [45]:
# Import necessary libraries
import pandas as pd
import django
from chembl_webresource_client.new_client import new_client

In [3]:
# Display available data entities
available_resources = [resource for resource in dir(new_client) if not resource.startswith('_')]
print(available_resources)

['activity', 'activity_supplementary_data_by_activity', 'assay', 'assay_class', 'atc_class', 'binding_site', 'biotherapeutic', 'cell_line', 'chembl_id_lookup', 'compound_record', 'compound_structural_alert', 'description', 'document', 'document_similarity', 'drug', 'drug_indication', 'drug_warning', 'go_slim', 'image', 'mechanism', 'metabolism', 'molecule', 'molecule_form', 'official', 'organism', 'protein_classification', 'similarity', 'source', 'substructure', 'target', 'target_component', 'target_relation', 'tissue', 'xref_source']


## Target Summary for Abeta ##

I want the Abeta protein from *Homo sapiens*, so I am specifying the target is the single protein (rather than a protein complex) from *Homo sapiens*. The **only** operator is especially useful for queries that result in many hits, as it is a faster API call that saves bandwidth

In [4]:
target = new_client.target
target_query = target.filter(pref_name__icontains=['Amyloid','Beta'], target_type='SINGLE PROTEIN', organism='Homo sapiens')
filtered_target_query = target_query.only(['target_chembl_id', 'organism', 'pref_name', 'target_type']) 
targets = pd.DataFrame.from_dict(filtered_target_query)
targets

,organism,pref_name,target_chembl_id,target_type
0,Homo sapiens,Beta amyloid A4 protein,CHEMBL2487,SINGLE PROTEIN
1,Homo sapiens,Endoplasmic reticulum-associated amyloid beta-...,CHEMBL4159,SINGLE PROTEIN
2,Homo sapiens,Amyloid beta-binding alcohol dehydrogenase,CHEMBL4295598,SINGLE PROTEIN


## Select appropriate target ##
#### For my case, target 0 is most appropriate ####

In [5]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL2487'

## Pull bioactivity data ##
The activity data is reported in different units of measurement (IC50, Ki, % Inhibition, etc.). Since Abeta is not an enzyme, the inhibition described is not referring to enzymatic activity, but instead is referring to inhibition of Abeta secretion (e.g. from BACE1), or ⍺β aggregation, or even Abeta oligomerization...
I'm comfortable lumping together amyloid aggregation and Abeta oligomerization, but I will exclude BACE1 inhibition.

In [223]:
activity = new_client.activity
bioact_filter = activity.get(target_chembl_id=selected_target)
#temp = bioact_filter.exclude(assay_description__icontains=['secreted','secretion','BACE1'])
bioact_df = pd.DataFrame.from_dict(bioact_filter)
bioact_df.head(10)

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,72652,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,10.3
1,None,None,73705,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,0.9
2,None,None,75935,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,2000.0
3,None,None,79258,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,1000.0
4,None,None,81617,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,242.0
5,None,None,88722,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,2000.0
6,None,None,93688,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,2000.0
7,None,None,94614,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,638.0
8,None,None,100600,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,339.0
9,None,None,104621,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,15.0


In [224]:
#How many samples?
print("Current # of samples", bioact_df.shape[0])

Current # of samples 8334


In [210]:
#Remove NAs
bioact_df2 = bioact_df[bioact_df.standard_value.notna()]

if 'No Data' in bioact_df['standard_value'].values:
    count_no_data = (bioact_df['standard_value'] == 'No Data').sum()
    print("Number of 'No Data' occurrences:", count_no_data)
else:
    print("'No Data' and 'NA' not found in the column.")

#"No Data" automatically removed after updates to Chembl_webresource_client?


'No Data' and 'NA' not found in the column.


## Drop samples where target is BACE1 ##

In [211]:
#remove empties
bioact_df2 = bioact_df2[bioact_df2.standard_value.notna()]

# List of keywords to filter out
keywords_to_remove = ['secreted', 'secretion', 'BACE1']

# Use mask for rows containing the keywords
mask = bioact_df2['assay_description'].str.contains('|'.join(keywords_to_remove), case=False)

# Create the new DataFrame by selecting rows that do not match the mask
df = bioact_df2[~mask]

#How many samples removed?
print("Non-BACE1 samples:", bioact_df2.shape[0])
print("# samples removed:", bioact_df.shape[0]-bioact_df2.shape[0])

# Display the filtered DataFrame
df.head(10)

Non-BACE1 samples: 6535
# samples removed: 1799


,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,72652,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,10.3
1,None,None,73705,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,0.9
2,None,None,75935,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,2000.0
3,None,None,79258,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,1000.0
4,None,None,81617,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,242.0
5,None,None,88722,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,2000.0
6,None,None,93688,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,2000.0
7,None,None,94614,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,638.0
8,None,None,100600,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,339.0
9,None,None,104621,[],CHEMBL649398,Inhibition constant against [125I]-7 (TZDM) bi...,B,None,None,BAO_0000192,...,Homo sapiens,Beta amyloid A4 protein,9606,None,None,Ki,nM,UO_0000065,None,15.0


## Drop "unreliable" samples ##

Certain samples are marked with a comment by the author. An IC$_{50}$ of 50000000 nM should be tagged with "Outside Typical Range" in the "Data Validity Comment" column. First let's see what akind of comments are already present with our data:

In [222]:
#list(df.columns)
df['standard_type'].value_counts()

Inhibition        3414
IC50              1285
Ki                 532
Activity           527
Potency            112
Kd                  97
FC                  80
EC50                73
Flu intensity       40
Imax                31
TIME                27
RFU                 22
Ratio               21
Ratio IC50          20
DC50                11
K                    9
Ka                   7
T1/2                 7
Tlag                 7
Stimulation          5
Bmax                 3
Ke(app)              2
Survival             2
Retention_time       2
Ratio Ki             1
INH                  1
Name: standard_type, dtype: int64

In [212]:
df['data_validity_comment'].value_counts()

Outside typical range    92
Name: data_validity_comment, dtype: int64

We have "None" (i.e. no comment necessary) and "Outside typical range". I will remove the samples marked "outside typical range.

In [213]:
df = df.drop(df[df['data_validity_comment'] == 'Outside typical range'].index)
count_OTR = df['data_validity_comment'].value_counts().get('Outside typical range', 0)
print("Number of OTRs:", count_OTR)

Number of OTRs: 0


Remove all samples measured in units other than K$_{i}$ and IC$_{50}$. 

In [ ]:
df_new = df[df['standard_type'].isin(['Ki', 'IC50'])]
print("Rows removed:", df.shape[0] - df_new.shape[0])
print("Rows remaining:", df_new.shape[0])
df = df_new

How many samples are measured in K$_{i}$ vs in IC$_{50}$?

In [230]:
value_counts = df['type'].value_counts()
count_ki = value_counts.get('Ki', 0)
count_ic50 = value_counts.get('IC50', 0)
percent_ki = ((count_ki)/(count_ki + count_ic50))*100
percent_ic50 = ((count_ic50)/(count_ki + count_ic50))*100
print(f"# samples reported in 'Ki': {count_ki} ({percent_ki:.0f}%)")
print(f"# samples reported in 'IC50': {count_ic50} ({percent_ic50:.0f}%)")

# samples reported in 'Ki': 508 (28%)
# samples reported in 'IC50': 1283 (72%)


Not unexpectedly, most are reported in IC50.

## Categorizing by activity ##

While K$_{i}$ is often calculated in undergraduate labs, it requires multiple assays of varying inhibitor concentrations to ascertain a specific K$_{i}$ for an inhibitor-enzyme pair. 
On the other hand, IC$_{50}$ varies with substrate concentration but requires only one assay to obtain a value, and is seen more often in published works. 
$$ IC_{50} = K_{i}(1 + {[S] \over K_{M}}) $$
<h6><center>Cheng-Prusoff equation  for  competitive  inhibition</center></h6>

K$_{i}$ and IC$_{50}$ can be interconverted using the appropriate Cheng-Prusoff equation, so long as the concentration of substrate and mode of inhibition are reported. ChEMBL does not include the assay conditions (i.e. substrate concentration) in an easily accessible format when pulled with SQL queries. However, Kalliokoski & Kramer (DOI: 0061007) proposed a conversion factor of ~2.3 activity units in order to bolster IC$_{50}$ data with K$_{i}$ data on ChEMBL.

I am categorizing molecules as being *highly active*, *active*, *intermediate* or *inactive* based on their IC$_{50}$ or K$_{i}$.

For IC$_{50}$ values:
* IC$_{50}$ < 100 nM = *highly active*.
* 100 nM < IC$_{50}$ < 1000 nM = *active*.
* 1000 nM < IC$_{50}$ < 10,000 nM = *intermediate*.
* IC$_{50}$ > 10,000 nM = *inactive*.

A K$_{i}$ conversion factor of 2.2426 will be applied. As such, for K$_{i}$ values:
* K$_{i}$ < 45 nM = *highly active*.
* 44.6684 nM < K$_{i}$ < 447 nM = *active*.
* 446.6836 nM < K$_{i}$ < 4,467 mM = *intermediate*.
* K$_{i}$ > 4,467 nM = *inactive*.

In [231]:
bioact_class = []
for i,j in df.iterrows():
    if j['standard_type'] == 'IC50':
        if float(j['standard_value']) <= 100:
            bioact_class.append("highly active")
        elif float(j['standard_value']) <= 1000:
            bioact_class.append("active")
        elif float(j['standard_value']) <= 10000:
            bioact_class.append("intermediate")
        else:
            bioact_class.append("inactive")
    elif j['standard_type'] == 'Ki':
        if float(j['standard_value']) <= 45:
            bioact_class.append("highly active")
        elif float(j['standard_value']) <= 447:
            bioact_class.append("active")
        elif float(j['standard_value']) <= 4467:
            bioact_class.append("intermediate")
        else:
            bioact_class.append("inactive")
#print(df.loc[:,('standard_type','standard_value')].head(10))   #For Troubleshooting
#print(bioact_class[:10])

  standard_type standard_value
0            Ki           10.3
1            Ki            0.9
2            Ki         2000.0
3            Ki         1000.0
4            Ki          242.0
5            Ki         2000.0
6            Ki         2000.0
7            Ki          638.0
8            Ki          339.0
9            Ki           15.0
['highly active', 'highly active', 'intermediate', 'intermediate', 'active', 'intermediate', 'intermediate', 'intermediate', 'active', 'highly active']
1817
1817


Generate new dataframe with only *molecule_chembl_id*, *canonical_smiles*, *standard_value* and the new *bioact_class* list.

In [244]:
df3 = df.loc[:,('molecule_chembl_id', 'canonical_smiles', 'standard_value')]
df3.loc[:,'bioact_class'] = bioact_class
print(df3.head(10))

  molecule_chembl_id                   canonical_smiles standard_value  \
0        CHEMBL81260  CN(C)c1ccc(-c2cn3cc(Br)ccc3n2)cc1           10.3   
1        CHEMBL55380   CN(C)c1ccc(-c2nc3ccc(I)cc3s2)cc1            0.9   
2        CHEMBL78410      CNc1ccc(-c2cn3cc(C)ccc3n2)cc1         2000.0   
3        CHEMBL81685         CNc1ccc(-c2cn3ccccc3n2)cc1         1000.0   
4        CHEMBL78621   Cc1ccc2nc(-c3ccc(N(C)C)cc3)cn2c1          242.0   
5       CHEMBL312622      CN(C)c1ccc(-c2cn3ccccc3n2)cc1         2000.0   
6       CHEMBL310444  CN(C)c1ccc(-c2nc3ccc(I)cn3c2I)cc1         2000.0   
7        CHEMBL78765      Cc1ccc2nc(-c3ccc(Br)cc3)cn2c1          638.0   
8       CHEMBL309920  CN(C)c1ccc2nc(-c3ccc(Br)cc3)cn2c1          339.0   
9        CHEMBL78012   CN(C)c1ccc(-c2cn3cc(I)ccc3n2)cc1           15.0   

    bioact_class  
0  highly active  
1  highly active  
2   intermediate  
3   intermediate  
4         active  
5   intermediate  
6   intermediate  
7   intermediate  
8         acti

In [ ]:
df3.to_csv('Abeta_bioactivity_preprocessed_data.csv', index=False)